# Table of Contents
<p>
<div class="lev1"><a href="#Data-from-the-Web"><span class="toc-item-num">1&nbsp;&nbsp;</span>Data from the Web</a></div>
<div class="lev1"><a href="#Getting-the-data"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Getting the data</a></div>
<div class="lev2"><a href="#Requesting-ISA-form"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Requesting ISA form</a></div>
<div class="lev2"><a href="#Finding-form-IDs"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Finding form IDs</a></div>
<div class="lev2"><a href="#Filtering-and-getting-the-data"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Filtering and getting the data</a></div>



# Data from the Web

In this homework we will extract interesting information from IS-Academia, the educational portal of EPFL. Specifically, we will focus on the part that allows public access to academic data. The list of registered students by section and semester is not offered as a downloadable dataset, so you will have to find a way to scrape the information we need. On this form you can select the data to download based on different criteria (e.g., year, semester, etc.)

You are not allowed to download manually all the tables -- rather you have to understand what parameters the server accepts, and generate accordingly the HTTP requests. For this task, Postman with the Interceptor extension can help you greatly. I recommend you to watch this brief tutorial to understand quickly how to use it. Your code in the iPython Notebook should not contain any hardcoded URL. To fetch the content from the IS-Academia server, you can use the Requests library with a Base URL, but all the other form parameters should be extracted from the HTML with BeautifulSoup. You can choose to download Excel or HTML files -- they both have pros and cons, as you will find out after a quick check. You can also choose to download data at different granularities (e.g., per semester, per year, etc.) but I recommend you not to download all the data in one shot because 1) the requests are likely to timeout and 2) we will overload the IS-Academia server.


In [794]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import glob
import requests
import re
from bs4 import BeautifulSoup
sns.set_context('notebook')

# Getting the data

## Finding ISA form 

The first part of the job in order to get the data is to get the parameters required to get the data we want.

In this purpose, we first do a get request on the ISA form with the link <http://isa.epfl.ch/imoniteur_ISAP/!GEDPUBLICREPORTS.filter?ww_i_reportModel=133685247>.

We also use BeautifulSoup on the resulting html response in order to parse it later.

In [795]:
r = requests.get('http://isa.epfl.ch/imoniteur_ISAP/!GEDPUBLICREPORTS.filter?ww_i_reportModel=133685247')
r.headers['content-type']
html_doc = r.text
isaForm = BeautifulSoup(html_doc, 'html.parser')

## Finding form IDs

Now that we've got the form's html code, we need to know which values of the form are used to filter and displayed the desired data. The values we're interested in are 'unité académique', 'période académique' and 'période pédagogique' (corresponding respectively to section, academic year and semester).

By inspecting the html code, we saw that the form items are 'option', it is then easy to get their value by using BeautifupSoup find and find_all method.

The following code will simply find the option value corresponding to section 'Informatique', and output it's value (the id used to filter the result).
```python
    soup.find('option', text = re.compile('Informatique'))['value']
```

We do the same thing for Bachelor 1st and 6th semester.
```python
    bachelor_1_id = soup.find('option', text = re.compile('Bachelor semestre 1'))['value']
    bachelor_6_id = soup.find('option', text = re.compile('Bachelor semestre 6'))['value']
```

And we get the academic years ids from 2007-2008 to 2016-2017 using a for loop (see in the cell below)

In [796]:
informatique_id = isaForm.find('option', text = re.compile('Informatique'))['value']
print("Id of informatique : ", informatique_id, "\n")

semester_ids = {}
for i in range(1, 7):
    semester_ids['Bachelor semestre ' + str(i)] = isaForm.find('option', text = re.compile('Bachelor semestre ' + str(i)))['value']
print("Id of Bachelor semester 1: ", semester_ids['Bachelor semestre 1'],"\n")
print("Id of Bachelor semester 6: ", semester_ids['Bachelor semestre 6'],"\n")

year_ids = {}
for y in range(2007, 2017):
    school_year = str(y) + "-" + str(y+1)
    year_ids[str(y) + "-" + str(y+1)] = [isaForm.find('option', text = re.compile(school_year))['value']]
    
print("years ids : (from 2007-2008 to 2016-2017)", year_ids)



Id of informatique :  249847 

Id of Bachelor semester 1:  249108 

Id of Bachelor semester 6:  942175 

years ids : (from 2007-2008 to 2016-2017) {'2010-2011': ['39486325'], '2016-2017': ['355925344'], '2008-2009': ['978187'], '2007-2008': ['978181'], '2013-2014': ['213637754'], '2009-2010': ['978195'], '2014-2015': ['213637922'], '2012-2013': ['123456101'], '2015-2016': ['213638028'], '2011-2012': ['123455150']}


## Filtering and getting the data

Now that we know the interesting IDs used in the form, we need to filter and request our data. For this purpose, we used Postman and Postman interceptor to intercept and inspect the request method used to get the data from the formula. 
  
</br>




The picture below shows all parameters used in the URL to filter and return results for:
* Section "Informatique"
* Academic period "2016-2017"
* Pedagogic period "Bachelor semestre 1"

<p>
    <img src="img/postman.png" alt="postman" align="center"/>
</p>

After playing a bit with the URL, we conclude that not all parameters were mandatory, the required parameters and their values are:

|parameter  | value |
|-----------|-------|
|ww_b_list  |must be '1'|  
|ww_i_reportmodel|must be '133685247'|
|ww_i_reportModelXsl|must be '133685270'|
|ww_x_UNITE_ACAD|correspond to the id of the section, taken from the form|
|ww_x_PERIODE_ACAD|correspond to the id of the academic year, taken from the form|
|ww_x_PERIODE_PEDAGO|correspond to the id of the semester, taken from the form|



In [797]:
def getFilteredPage(academic_year, semester):
    params = {'ww_b_list':'1',
            'ww_i_reportmodel':'133685247',
            'ww_i_reportModelXsl':'133685270',
            'ww_x_UNITE_ACAD':informatique_id,
            'ww_x_PERIODE_ACAD':year_ids[academic_year],
            'ww_x_PERIODE_PEDAGO':semester_ids[semester]}
    r = requests.get('http://isa.epfl.ch/imoniteur_ISAP/!GEDPUBLICREPORTS.filter?', params)
    html_doc = r.text
    return BeautifulSoup(html_doc, 'html.parser'), params

def getResultPage(academic_year, semester):
    filteredPage, params = getFilteredPage(academic_year, semester)
    params['ww_x_GPS'] = filteredPage.find_all('a')[1].get('onclick').split("ww_x_GPS=")[1].split("')")[0]
    r = requests.get('http://isa.epfl.ch/imoniteur_ISAP/!GEDPUBLICREPORTS.bhtml?', params)
    return BeautifulSoup(r.text, 'html.parser')

def getResultAllYears(semester):
    global_table = []
    for year_id in year_ids:
        soup = getResultPage(year_id, semester)

        students_tr = soup.body.hr.table.find_all('tr')[2:]
        students = []
        for i in range (0,len(students_tr)):
            student = students_tr[i].find_all('td')
            students.append([student[0].text,student[1].text.replace(u'\xa0', u' '),student[7].text,student[10].text])

        pd_student = pd.DataFrame(students, columns=['Gender', 'Name', 'Status_' + year_id, 'Sciper'])

        global_table.append(pd_student)
    return global_table

def joinTables(global_table):
    joined_table = global_table[0]
    for single_table in global_table[1:len(global_table)]:
        joined_table = pd.merge(joined_table, single_table, how='outer', on=['Gender','Name','Sciper'])
    return joined_table
  
def formatTable(joined_table_bch):
    bch_no_string = joined_table_bch.drop(joined_table_bch.columns[[0,1,3]], axis=1)
    bch = bch_no_string.sort_index(axis=1)
    
    year = 2007
    for bch_col in bch:
        bch[bch_col] = bch[bch_col].replace('Présent', year)
        year = year + 1
    return bch
 
    
    
    

In [798]:
global_table_bch1 = getResultAllYears('Bachelor semestre 1')

In [799]:
joined_table_bch1 = joinTables(global_table_bch1)
joined_table_bch1.head()

,Gender,Name,Status_2010-2011,Sciper,Status_2016-2017,Status_2008-2009,Status_2007-2008,Status_2013-2014,Status_2009-2010,Status_2014-2015,Status_2012-2013,Status_2015-2016,Status_2011-2012
0,Monsieur,Achour Maher Ali,Présent,202124,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Monsieur,Aiulfi Loris Sandro,Présent,202293,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Présent
2,Monsieur,Akiba David,Présent,206418,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Présent
3,Madame,Al Azawi Marwa,Présent,195766,NaN,NaN,NaN,NaN,Présent,NaN,NaN,NaN,NaN
4,Monsieur,Albasini Romain,Présent,198197,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Présent


In [800]:
joined_table_bch1.shape

(1323, 13)

In [801]:
table_no_conge_bch1 = joined_table_bch1.replace('Congé', np.nan).replace('Attente', np.nan).dropna(thresh=4)
table_no_conge_bch1.shape

(1323, 13)

In [802]:
formated_table_bch1 = formatTable(table_no_conge_bch1)
bch1_min = formated_table_bch1.min(axis=1)
bch1_min.head()

0    2010.0
1    2010.0
2    2010.0
3    2009.0
4    2010.0
dtype: float64

In [803]:
formated_table_bch1 = formatTable(table_no_conge_bch1)
bch1_mim = formated_table_bch1.min(axis=1)
bch1_min.head()

0    2010.0
1    2010.0
2    2010.0
3    2009.0
4    2010.0
dtype: float64

In [804]:
bch1_string = table_no_conge_bch1.drop(table_no_conge_bch1.columns[[1,2,4,5,6,7,8,9,10,11,12]], axis=1)
bch1_final = pd.merge(bch1_string, bch1_min.to_frame(),left_index=True, right_index=True)
bch1_final.columns = [['Gender', 'Sciper', 'Start_year']]
bch1_final.head()

,Gender,Sciper,Start_year
0,Monsieur,202124,2010.0
1,Monsieur,202293,2010.0
2,Monsieur,206418,2010.0
3,Madame,195766,2009.0
4,Monsieur,198197,2010.0


In [805]:
global_table_bch6 = getResultAllYears('Bachelor semestre 6')

In [806]:
joined_table_bch6 = joinTables(global_table_bch6)
joined_table_bch6.head()

,Gender,Name,Status_2010-2011,Sciper,Status_2016-2017,Status_2008-2009,Status_2007-2008,Status_2013-2014,Status_2009-2010,Status_2014-2015,Status_2012-2013,Status_2015-2016,Status_2011-2012
0,Monsieur,Arnfred Jonas,Congé,184772,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Monsieur,Aubelle Flavien,Présent,174905,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Présent
2,Monsieur,Baeriswyl Jonathan,Présent,179406,NaN,NaN,NaN,NaN,Présent,NaN,NaN,NaN,NaN
3,Monsieur,Barroco Michael,Présent,179428,NaN,NaN,NaN,NaN,Présent,NaN,NaN,NaN,NaN
4,Monsieur,Bloch Remi,Présent,179426,NaN,NaN,NaN,NaN,Présent,NaN,NaN,NaN,NaN


In [807]:
joined_table_bch6.shape

(516, 13)

In [808]:
table_no_conge_bch6 = joined_table_bch6.replace('Congé', np.nan).replace('Attente', np.nan).dropna(thresh=4)
table_no_conge_bch6.shape

(400, 13)

In [809]:
#table_no_conge_bch6 = table_no_conge_bch6.fillna(0)
formated_table_bch6 = formatTable(table_no_conge_bch6)
bch6_max = formated_table_bch6.max(axis=1)
bch6_max.head()

1    2011.0
2    2010.0
3    2010.0
4    2010.0
5    2010.0
dtype: float64

In [810]:
bch6_string = table_no_conge_bch6.drop(table_no_conge_bch6.columns[[1,2,4,5,6,7,8,9,10,11,12]], axis=1)
bch6_final = pd.merge(bch6_string, bch6_max.to_frame(),left_index=True, right_index=True)
bch6_final.columns = [['Gender', 'Sciper', 'Last_year']]
bch6_final.head()


,Gender,Sciper,Last_year
1,Monsieur,174905,2011.0
2,Monsieur,179406,2010.0
3,Monsieur,179428,2010.0
4,Monsieur,179426,2010.0
5,Monsieur,185949,2010.0


In [811]:
bachelor_survivors = pd.merge(bch1_final, bch6_final, how='inner', on=['Gender','Sciper'])
bachelor_survivors.head()

,Gender,Sciper,Start_year,Last_year
0,Monsieur,202293,2010.0,2014.0
1,Monsieur,201284,2010.0,2012.0
2,Monsieur,194182,2009.0,2012.0
3,Monsieur,185991,2010.0,2013.0
4,Monsieur,204958,2010.0,2013.0


In [812]:
bachelor_survivors.shape

(297, 4)

In [813]:
bachelor_duration = bachelor_survivors.Last_year - bachelor_survivors.Start_year + 1
bachelor_duration.head()

0    5.0
1    3.0
2    4.0
3    4.0
4    4.0
dtype: float64

In [814]:
gender_stats = pd.merge(bachelor_survivors[['Gender']], bachelor_duration.to_frame(), left_index=True, right_index=True)
gender_stats.columns = [['Gender', 'Bachelor duration']]
gender_stats.head()

,Gender,Bachelor duration
0,Monsieur,5.0
1,Monsieur,3.0
2,Monsieur,4.0
3,Monsieur,4.0
4,Monsieur,4.0


In [815]:
gender_stats.groupby('Gender').mean()

,Bachelor duration
Gender,
Madame,3.291667
Monsieur,3.593407


In [816]:
gender_stats.describe()

,Bachelor duration
count,297.000000
mean,3.569024
std,0.764135
min,3.000000
25%,3.000000
50%,3.000000
75%,4.000000
max,7.000000


In [817]:
gender_stats

,Gender,Bachelor duration
0,Monsieur,5.0
1,Monsieur,3.0
2,Monsieur,4.0
3,Monsieur,4.0
4,Monsieur,4.0
5,Monsieur,3.0
6,Monsieur,4.0
7,Monsieur,3.0
8,Monsieur,4.0
9,Monsieur,5.0
